In [ ]:
import getpass
import os

# Set Google API key
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


In [ ]:

# Import relevant functionality
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("gemini-2.0-flash", model_provider="google-genai")  # Corrected model and provider
tools = []  # Define your tools here, if any
agent_executor = create_react_agent(model, tools, checkpointer=memory)


In [ ]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///real_estate.db")

print(db.get_usable_table_names())
print(db.run("PRAGMA table_info(properties);"))

['properties']
[(0, 'id', 'INTEGER', 0, None, 1), (1, 'title', 'TEXT', 0, None, 0), (2, 'location', 'TEXT', 0, None, 0), (3, 'price', 'INTEGER', 0, None, 0), (4, 'bedrooms', 'INTEGER', 0, None, 0), (5, 'bathrooms', 'INTEGER', 0, None, 0), (6, 'area', 'INTEGER', 0, None, 0), (7, 'property_type', 'TEXT', 0, None, 0)]


In [22]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x79a56a50bce0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x79a56a50bce0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x79a56a50bce0>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [23]:
system_message = """
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect="SQLite",
    top_k=5,
)

In [24]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools, prompt=system_message)

In [25]:
question = "Give me the name of properties under 300000 near Austin, TX."

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Give me the name of properties under 300000 near Austin, TX.
================================== Ai Message ==================================

I need to know what tables are available in the database before I can formulate a query.
Tool Calls:
  sql_db_list_tables (f6592a14-a2f3-4097-8805-14015eefd0fb)
 Call ID: f6592a14-a2f3-4097-8805-14015eefd0fb
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

properties
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (005d47a2-4c84-45e9-975a-8ecc473bebb6)
 Call ID: 005d47a2-4c84-45e9-975a-8ecc473bebb6
  Args:
    table_names: properties
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE properties (
	id INTEGER, 
	title TEXT, 
	location TEXT, 
	price INTEGER, 
	bedrooms INTEGER, 